<a href="https://colab.research.google.com/github/HiagoAD/GDI2/blob/master/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [4]:
!pip install mlflow --quiet
get_ipython().system_raw("mlflow ui --port 5000 &")

!pip install pyngrok --quiet

from pyngrok import ngrok
from getpass import getpass

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = getpass('Enter the ngrok authtoken: ')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

import mlflow

Enter the ngrok authtoken: ··········
MLflow Tracking UI: https://b11d88cac023.ngrok.io


## Load datasets


In [5]:
dengue = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2020.csv", encoding ="UTF-8")
emlurb = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2020.csv", encoding ="UTF-8")

dengue19 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2019.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb19 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2019.csv", encoding ="UTF-8")

dengue18 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2018.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb18 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2018.csv", encoding ="UTF-8")

dengue17 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2017.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb17 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2017.csv", encoding ="UTF-8")

dengue16 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2016.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb16 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2016.csv", encoding ="UTF-8")


b'Skipping line 108: expected 126 fields, saw 127\nSkipping line 1485: expected 126 fields, saw 127\nSkipping line 2584: expected 126 fields, saw 127\nSkipping line 4415: expected 126 fields, saw 127\nSkipping line 5659: expected 126 fields, saw 127\nSkipping line 6469: expected 126 fields, saw 127\n'
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (55,113) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Dados de 2020 - 2016


Iremos fazer os mesmos tratamentos anteriores para os dados de 2019 até 2016.

In [6]:
# Dados Dengue

merge_dengue = dengue.append([dengue19,dengue18,dengue17,dengue16], ignore_index = True)
merge_dengue = merge_dengue[["febre", "mialgia",'vomito', "dt_notificacao",'nu_idade','nausea','cefaleia','dor_costas','diabetes','renal']]
merge_dengue['dt_notificacao'] = pd.to_datetime(merge_dengue['dt_notificacao'])

# Agrupamos por número de casos por dia

#merge_dengue = merge_dengue.groupby([pd.Grouper(key="dt_notificacao", freq="D")]).size().reset_index(name="CASOS_DENGUE")


# Dados Emlurb

merge_emlurb = emlurb.append([emlurb19,emlurb18,emlurb17,emlurb16], ignore_index = True)
merge_emlurb = merge_emlurb[["GRUPOSERVICO_CODIGO", "DATA_DEMANDA"]]
merge_emlurb = merge_emlurb[(merge_emlurb.GRUPOSERVICO_CODIGO == 11)]
merge_emlurb["DATA_DEMANDA"] = pd.to_datetime(merge_emlurb["DATA_DEMANDA"],errors='coerce')

# Agrupamos por número de solicitações por dia

#merge_emlurb = merge_emlurb.groupby([pd.Grouper(key="DATA_DEMANDA", freq="D")]).size().reset_index(name="CHAMAD0S_EMLURB")

#year_filter = merge_emlurb[(merge_emlurb['DATA_DEMANDA'].dt.year == 2018)]


merge_dengue = merge_dengue.dropna()
merge_dengue['febre'] = merge_dengue['febre'].astype('int')
merge_dengue['mialgia'] = merge_dengue['mialgia'].astype('int')
merge_dengue['vomito'] = merge_dengue['vomito'].astype('int')
merge_dengue['nausea'] = merge_dengue['nausea'].astype('int')
merge_dengue['cefaleia'] = merge_dengue['cefaleia'].astype('int')
merge_dengue['dor_costas'] = merge_dengue['dor_costas'].astype('int')
merge_dengue['diabetes'] = merge_dengue['diabetes'].astype('int')
merge_dengue['renal'] = merge_dengue['renal'].astype('int')
merge_dengue = merge_dengue.sort_values('dt_notificacao')
merge_dengue

,febre,mialgia,vomito,dt_notificacao,nu_idade,nausea,cefaleia,dor_costas,diabetes,renal
14367,1,1,2,2016-01-03,4024,2,2,2,2,2
13813,2,2,2,2016-01-03,4059,2,2,2,2,2
13658,1,1,2,2016-01-04,4017,1,1,2,2,2
13595,1,1,2,2016-01-04,4031,2,1,2,2,2
13781,1,1,2,2016-01-04,4054,2,1,2,2,2
...,...,...,...,...,...,...,...,...,...,...
975,1,1,1,2020-07-16,4008,1,1,2,2,2
1007,1,1,2,2020-07-17,4017,2,1,2,2,2
1005,1,2,2,2020-07-19,4010,2,2,2,2,2
1003,1,1,2,2020-07-20,3001,2,2,2,2,2


## Feature de predição

1.   Número de casos de dengue por dia.
2.   Número de solicitações de drenagem por dia.



## Separação de dados: Treino, Validação e Teste

Os dados estão em ordem temporal, treino usa os primeiros 2/4 dos dados, em seguida 1/4 pra validação e 1/4 para testes.

In [7]:
def split_values(df, column_name):
  X = df.drop(columns=[column_name,'dt_notificacao']).values
  y = df[column_name].values

  return X, y

In [8]:
features_results = merge_dengue.drop(columns=['febre','dt_notificacao']).values
labels_results = merge_dengue['febre'].values

feature_train, feature_test, label_train, label_test = train_test_split(features_results, labels_results, test_size=0.25, shuffle=False)
feature_train, feature_validation, label_train, label_validation =  train_test_split(feature_train, label_train, test_size=(1/3), shuffle=False)

print(features_results.shape)
print(feature_train.shape)
print(feature_test.shape)
print(feature_validation.shape)

(25913, 8)
(12956, 8)
(6479, 8)
(6478, 8)


In [9]:
# train, validation, test = np.split(merge_dengue.sample(frac=1, random_state=42), [int(.6*len(merge_dengue)), int(.8*len(merge_dengue))])

# column_name = 'febre'

# X_train, y_train = split_values(train, column_name)
# X_valid, y_valid = split_values(validation, column_name)
# X_test,  y_test  = split_values(test, column_name)

X_train, y_train = feature_train, label_train
X_valid, y_valid = feature_validation, label_validation
X_test, y_test = feature_test, label_test

In [10]:
print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)

(12956, 8)
(6479, 8)
(6478, 8)


## Escolha de Algoritmos

*   Decision Tree
*   Random Forest
*   Naive Bayes
*   MLP
*   KNN



In [11]:
decision_tree = DecisionTreeClassifier()
random_forest = RandomForestClassifier()
naive_bayes   = GaussianNB()
ml_perceptron = MLPClassifier()
knn = KNeighborsClassifier()

decision_tree.fit(X_train, y_train)
y_decision_tree_train = decision_tree.predict(X_train)
y_decision_tree_test  = decision_tree.predict(X_test)

random_forest.fit(X_train, y_train)
y_random_forest_train = random_forest.predict(X_train)
y_random_forest_test  = random_forest.predict(X_test)

naive_bayes.fit(X_train, y_train)
y_naive_bayes_train = naive_bayes.predict(X_train)
y_naive_bayes_test  = naive_bayes.predict(X_test)

ml_perceptron.fit(X_train, y_train)
y_ml_perceptron_train = ml_perceptron.predict(X_train)
y_ml_perceptron_test  = ml_perceptron.predict(X_test)

knn.fit(X_train, y_train)
y_knn_train = knn.predict(X_train)
y_knn_test = knn.predict(X_test)

In [12]:
print("=== Precisão Testes === \n\n")

print("Precisão Decision Tree:", accuracy_score(y_test, y_decision_tree_test))
print("Precisão Random Forest:", accuracy_score(y_test, y_random_forest_test))
print("Precisão Naive Bayes:" , accuracy_score(y_test, y_naive_bayes_test))
print("Precisão MLP: ", accuracy_score(y_test, y_ml_perceptron_test))
print("Precisão KNN: ", accuracy_score(y_test, y_knn_test))

print("\n\n=== Precisão Treino === \n\n")

print("Precisão Decision Tree:", accuracy_score(y_train, y_decision_tree_train))
print("Precisão Random Forest:", accuracy_score(y_train, y_random_forest_train))
print("Precisão Naive Bayes:" , accuracy_score(y_train, y_naive_bayes_train))
print("Precisão MLP: ", accuracy_score(y_train, y_ml_perceptron_train))
print("Precisão KNN: ", accuracy_score(y_train, y_knn_train))

=== Precisão Testes === 


Precisão Decision Tree: 0.8793023614755363
Precisão Random Forest: 0.8786849822503473
Precisão Naive Bayes: 0.735144312393888
Precisão MLP:  0.8972063590060194
Precisão KNN:  0.880845809538509


=== Precisão Treino === 


Precisão Decision Tree: 0.8809817845013893
Precisão Random Forest: 0.8809817845013893
Precisão Naive Bayes: 0.7185087990120408
Precisão MLP:  0.7540907687557888
Precisão KNN:  0.8589070700833591


## MLFlow e Hiper-parâmetros

Nessa etapa, adicionaremos o MLFlow e o Optuna para seleção de hiper-parâmetros sobre o conjunto de validação.

In [14]:
from optuna.integration.mlflow import MLflowCallback
mlflc = MLflowCallback(
    tracking_uri=mlflow.get_tracking_uri(),
    metric_name="Accuracy",
    
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: ExperimentalWarning:

MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.



### Decision Tree

In [15]:
@mlflc.track_in_mlflow()
def objective_decisionTree(trial):
    max_depth = trial.suggest_int("max_depth", 1, 15, 1)
    split     = trial.suggest_int('min_samples_split', 3, 15, 1)
    leaf      = trial.suggest_int('min_samples_leaf', 3, 15, 1)

    decisionTree_selection = DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=leaf, min_samples_split=split)
    decisionTree_selection.fit(X_train, y_train)
    
    prediction_validation = decisionTree_selection.predict(X_valid)
    accuracy_validation = accuracy_score(y_valid, prediction_validation)

    return accuracy_validation

study_dt = optuna.create_study(direction="maximize", study_name="Decision Tree")
mlflow.end_run()
study_dt.optimize(objective_decisionTree, n_trials=50, callbacks=[mlflc])
print(study_dt.best_trial)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ExperimentalWarning:

track_in_mlflow is experimental (supported from v2.9.0). The interface can change in the future.

[I 2021-08-15 21:33:43,661] A new study created in memory with name: ASD
[I 2021-08-15 21:33:43,781] Trial 0 finished with value: 0.8965730163630751 and parameters: {'max_depth': 10, 'min_samples_split': 9, 'min_samples_leaf': 15}. Best is trial 0 with value: 0.8965730163630751.
[I 2021-08-15 21:33:43,819] Trial 1 finished with value: 0.8970361222599568 and parameters: {'max_depth': 6, 'min_samples_split': 4, 'min_samples_leaf': 8}. Best is trial 1 with value: 0.8970361222599568.
[I 2021-08-15 21:33:43,855] Trial 2 finished with value: 0.8965730163630751 and parameters: {'max_depth': 12, 'min_samples_split': 3, 'min_samples_leaf': 10}. Best is trial 1 with value: 0.8970361222599568.


INFO: 'ASD' does not exist. Creating a new experiment


[I 2021-08-15 21:33:43,896] Trial 3 finished with value: 0.8962642790984872 and parameters: {'max_depth': 11, 'min_samples_split': 12, 'min_samples_leaf': 7}. Best is trial 1 with value: 0.8970361222599568.
[I 2021-08-15 21:33:43,936] Trial 4 finished with value: 0.8962642790984872 and parameters: {'max_depth': 15, 'min_samples_split': 10, 'min_samples_leaf': 6}. Best is trial 1 with value: 0.8970361222599568.
[I 2021-08-15 21:33:43,974] Trial 5 finished with value: 0.891015745600494 and parameters: {'max_depth': 4, 'min_samples_split': 8, 'min_samples_leaf': 10}. Best is trial 1 with value: 0.8970361222599568.
[I 2021-08-15 21:33:44,013] Trial 6 finished with value: 0.8956468045693116 and parameters: {'max_depth': 9, 'min_samples_split': 11, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.8970361222599568.
[I 2021-08-15 21:33:44,053] Trial 7 finished with value: 0.8965730163630751 and parameters: {'max_depth': 15, 'min_samples_split': 8, 'min_samples_leaf': 10}. Best is trial 1 

FrozenTrial(number=16, values=[0.8971904908922507], datetime_start=datetime.datetime(2021, 8, 15, 21, 33, 44, 378156), datetime_complete=datetime.datetime(2021, 8, 15, 21, 33, 44, 402857), params={'max_depth': 8, 'min_samples_split': 6, 'min_samples_leaf': 8}, distributions={'max_depth': IntUniformDistribution(high=15, low=1, step=1), 'min_samples_split': IntUniformDistribution(high=15, low=3, step=1), 'min_samples_leaf': IntUniformDistribution(high=15, low=3, step=1)}, user_attrs={}, system_attrs={'mlflow_run_id': '920cbbe2bb124babbdf23d191060daff'}, intermediate_values={}, trial_id=16, state=TrialState.COMPLETE, value=None)


In [16]:
print(study_dt.best_trial.value)
print(study_dt.best_trial.params)

0.8971904908922507
{'max_depth': 8, 'min_samples_split': 6, 'min_samples_leaf': 8}


### Random Forest

In [17]:
def objective_randomForest(trial):
    forrest_n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    forrest_criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    forrest_max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])

    randomForrest_selection = RandomForestClassifier(n_estimators=forrest_n_estimators, criterion=forrest_criterion, max_features=forrest_max_features)
    randomForrest_selection.fit(X_train, y_train) 

    prediction_validation = randomForrest_selection.predict(X_valid)
    accuracy_validation = accuracy_score(y_valid, prediction_validation)

    return accuracy_validation

study_rf = optuna.create_study(direction="maximize", study_name="Random Forest")
mlflow.end_run()
study_rf.optimize(objective_randomForest, n_trials=50, callbacks=[mlflc])
print(study_rf.best_trial)

[I 2021-08-15 21:38:22,658] A new study created in memory with name: Random Forest
[I 2021-08-15 21:38:25,319] Trial 0 finished with value: 0.8913244828650818 and parameters: {'n_estimators': 260, 'criterion': 'entropy', 'max_features': 'sqrt'}. Best is trial 0 with value: 0.8913244828650818.


INFO: 'Random Forest' does not exist. Creating a new experiment


[I 2021-08-15 21:38:31,015] Trial 1 finished with value: 0.8916332201296696 and parameters: {'n_estimators': 609, 'criterion': 'gini', 'max_features': 'log2'}. Best is trial 1 with value: 0.8916332201296696.
[I 2021-08-15 21:38:40,593] Trial 2 finished with value: 0.8914788514973757 and parameters: {'n_estimators': 939, 'criterion': 'entropy', 'max_features': 'log2'}. Best is trial 1 with value: 0.8916332201296696.
[I 2021-08-15 21:38:46,647] Trial 3 finished with value: 0.8916332201296696 and parameters: {'n_estimators': 679, 'criterion': 'gini', 'max_features': 'sqrt'}. Best is trial 1 with value: 0.8916332201296696.
[I 2021-08-15 21:38:50,443] Trial 4 finished with value: 0.8916332201296696 and parameters: {'n_estimators': 369, 'criterion': 'entropy', 'max_features': 'log2'}. Best is trial 1 with value: 0.8916332201296696.
[I 2021-08-15 21:38:55,428] Trial 5 finished with value: 0.8920963260265514 and parameters: {'n_estimators': 508, 'criterion': 'entropy', 'max_features': 'auto'}.

FrozenTrial(number=5, values=[0.8920963260265514], datetime_start=datetime.datetime(2021, 8, 15, 21, 38, 50, 464775), datetime_complete=datetime.datetime(2021, 8, 15, 21, 38, 55, 427779), params={'n_estimators': 508, 'criterion': 'entropy', 'max_features': 'auto'}, distributions={'n_estimators': IntUniformDistribution(high=1000, low=100, step=1), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'max_features': CategoricalDistribution(choices=('auto', 'sqrt', 'log2'))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=5, state=TrialState.COMPLETE, value=None)


In [18]:
print(study_rf.best_trial.value)
print(study_rf.best_trial.params)

0.8920963260265514
{'n_estimators': 508, 'criterion': 'entropy', 'max_features': 'auto'}


### Naive Bayes

In [19]:
def objective_naiveBayes(trial):
    nb_var_smoothing = trial.suggest_float('var_smoothing', 1e-9, 1e9, log=True)

    naiveBayes_selection = GaussianNB(var_smoothing=nb_var_smoothing)
    naiveBayes_selection.fit(X_train, y_train)     

    prediction_validation = naiveBayes_selection.predict(X_valid)
    accuracy_validation = accuracy_score(y_valid, prediction_validation)

    return accuracy_validation

study_nb = optuna.create_study(direction="maximize", study_name="Neive Bayes")
mlflow.end_run()
study_nb.optimize(objective_naiveBayes, n_trials=50, callbacks=[mlflc])
print(study_nb.best_trial)

[I 2021-08-15 21:44:01,358] A new study created in memory with name: Neive Bayes
[I 2021-08-15 21:44:01,378] Trial 0 finished with value: 0.8672429762272307 and parameters: {'var_smoothing': 6199.49291075694}. Best is trial 0 with value: 0.8672429762272307.
[I 2021-08-15 21:44:01,408] Trial 1 finished with value: 0.8672429762272307 and parameters: {'var_smoothing': 1616174.9364794856}. Best is trial 0 with value: 0.8672429762272307.
[I 2021-08-15 21:44:01,434] Trial 2 finished with value: 0.8672429762272307 and parameters: {'var_smoothing': 37.50502674308528}. Best is trial 0 with value: 0.8672429762272307.
[I 2021-08-15 21:44:01,459] Trial 3 finished with value: 0.8672429762272307 and parameters: {'var_smoothing': 0.0055743379014353755}. Best is trial 0 with value: 0.8672429762272307.
[I 2021-08-15 21:44:01,485] Trial 4 finished with value: 0.8672429762272307 and parameters: {'var_smoothing': 0.0002662308216903886}. Best is trial 0 with value: 0.8672429762272307.
[I 2021-08-15 21:44:0

INFO: 'Neive Bayes' does not exist. Creating a new experiment


[I 2021-08-15 21:44:01,586] Trial 8 finished with value: 0.8672429762272307 and parameters: {'var_smoothing': 2.3437422419670653e-05}. Best is trial 0 with value: 0.8672429762272307.
[I 2021-08-15 21:44:01,619] Trial 9 finished with value: 0.8672429762272307 and parameters: {'var_smoothing': 285.81567455809216}. Best is trial 0 with value: 0.8672429762272307.
[I 2021-08-15 21:44:01,647] Trial 10 finished with value: 0.8672429762272307 and parameters: {'var_smoothing': 495453407.3195298}. Best is trial 0 with value: 0.8672429762272307.
[I 2021-08-15 21:44:01,674] Trial 11 finished with value: 0.8672429762272307 and parameters: {'var_smoothing': 703153484.2152555}. Best is trial 0 with value: 0.8672429762272307.
[I 2021-08-15 21:44:01,701] Trial 12 finished with value: 0.7545538746526705 and parameters: {'var_smoothing': 7.805851603121682e-09}. Best is trial 0 with value: 0.8672429762272307.
[I 2021-08-15 21:44:01,728] Trial 13 finished with value: 0.8672429762272307 and parameters: {'va

FrozenTrial(number=0, values=[0.8672429762272307], datetime_start=datetime.datetime(2021, 8, 15, 21, 44, 1, 367227), datetime_complete=datetime.datetime(2021, 8, 15, 21, 44, 1, 378154), params={'var_smoothing': 6199.49291075694}, distributions={'var_smoothing': LogUniformDistribution(high=1000000000.0, low=1e-09)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=0, state=TrialState.COMPLETE, value=None)


In [20]:
print(study_nb.best_trial.value)
print(study_nb.best_trial.params)

0.8672429762272307
{'var_smoothing': 6199.49291075694}


### MLP

In [21]:
def objective_MLP(trial):
    layer_size = trial.suggest_int('hidden_layer_sizes', 10, 1000, 10)
    activation = trial.suggest_categorical('activation', ['logistic', 'relu', 'tanh'])
    solver = trial.suggest_categorical('solver', ['adam', 'sgd'])
    mlp_selection = MLPClassifier(layer_size, activation=activation, solver=solver)
    mlp_selection.fit(X_train, y_train)     

    prediction_validation = mlp_selection.predict(X_valid)
    accuracy_validation = accuracy_score(y_valid, prediction_validation)

    return accuracy_validation

study_mlp = optuna.create_study(direction="maximize", study_name="MLP")
mlflow.end_run()
study_mlp.optimize(objective_MLP, n_trials=50, callbacks=[mlflc])
print(study_mlp.best_trial)

[I 2021-08-15 21:44:02,818] A new study created in memory with name: MLP
[I 2021-08-15 21:44:10,428] Trial 0 finished with value: 0.8672429762272307 and parameters: {'hidden_layer_sizes': 440, 'activation': 'tanh', 'solver': 'adam'}. Best is trial 0 with value: 0.8672429762272307.


INFO: 'MLP' does not exist. Creating a new experiment


[I 2021-08-15 21:44:11,527] Trial 1 finished with value: 0.8672429762272307 and parameters: {'hidden_layer_sizes': 70, 'activation': 'logistic', 'solver': 'adam'}. Best is trial 0 with value: 0.8672429762272307.
[I 2021-08-15 21:44:15,702] Trial 2 finished with value: 0.8672429762272307 and parameters: {'hidden_layer_sizes': 320, 'activation': 'relu', 'solver': 'sgd'}. Best is trial 0 with value: 0.8672429762272307.
[I 2021-08-15 21:44:22,088] Trial 3 finished with value: 0.8672429762272307 and parameters: {'hidden_layer_sizes': 620, 'activation': 'logistic', 'solver': 'sgd'}. Best is trial 0 with value: 0.8672429762272307.
[I 2021-08-15 21:44:31,960] Trial 4 finished with value: 0.8672429762272307 and parameters: {'hidden_layer_sizes': 930, 'activation': 'logistic', 'solver': 'adam'}. Best is trial 0 with value: 0.8672429762272307.
[I 2021-08-15 21:44:43,669] Trial 5 finished with value: 0.8672429762272307 and parameters: {'hidden_layer_sizes': 730, 'activation': 'tanh', 'solver': 'ad

FrozenTrial(number=16, values=[0.8673973448595246], datetime_start=datetime.datetime(2021, 8, 15, 21, 45, 26, 93838), datetime_complete=datetime.datetime(2021, 8, 15, 21, 45, 30, 265818), params={'hidden_layer_sizes': 390, 'activation': 'logistic', 'solver': 'adam'}, distributions={'hidden_layer_sizes': IntUniformDistribution(high=1000, low=10, step=10), 'activation': CategoricalDistribution(choices=('logistic', 'relu', 'tanh')), 'solver': CategoricalDistribution(choices=('adam', 'sgd'))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=16, state=TrialState.COMPLETE, value=None)


In [22]:
print(study_mlp.best_trial.value)
print(study_mlp.best_trial.params)

0.8673973448595246
{'hidden_layer_sizes': 390, 'activation': 'logistic', 'solver': 'adam'}


### KNN

In [23]:
def objective_KNN(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 5 , 500)
    knn_selection = KNeighborsClassifier(n_neighbors)
    knn_selection.fit(X_train, y_train)     

    prediction_validation = knn_selection.predict(X_valid)
    accuracy_validation = accuracy_score(y_valid, prediction_validation)

    return accuracy_validation

study_knn = optuna.create_study(direction="maximize", study_name="KNN")
mlflow.end_run()
study_knn.optimize(objective_KNN, n_trials=50, callbacks=[mlflc])
print(study_knn.best_trial)

[I 2021-08-15 21:49:16,378] A new study created in memory with name: KNN
[I 2021-08-15 21:49:17,026] Trial 0 finished with value: 0.8859215807347947 and parameters: {'n_neighbors': 217}. Best is trial 0 with value: 0.8859215807347947.


INFO: 'KNN' does not exist. Creating a new experiment


[I 2021-08-15 21:49:17,770] Trial 1 finished with value: 0.8757332510033962 and parameters: {'n_neighbors': 273}. Best is trial 0 with value: 0.8859215807347947.
[I 2021-08-15 21:49:18,833] Trial 2 finished with value: 0.8672429762272307 and parameters: {'n_neighbors': 441}. Best is trial 0 with value: 0.8859215807347947.
[I 2021-08-15 21:49:19,376] Trial 3 finished with value: 0.8862303179993826 and parameters: {'n_neighbors': 153}. Best is trial 3 with value: 0.8862303179993826.
[I 2021-08-15 21:49:19,949] Trial 4 finished with value: 0.8870021611608521 and parameters: {'n_neighbors': 161}. Best is trial 4 with value: 0.8870021611608521.
[I 2021-08-15 21:49:20,731] Trial 5 finished with value: 0.8728002469898116 and parameters: {'n_neighbors': 281}. Best is trial 4 with value: 0.8870021611608521.
[I 2021-08-15 21:49:21,473] Trial 6 finished with value: 0.8738808274158691 and parameters: {'n_neighbors': 276}. Best is trial 4 with value: 0.8870021611608521.
[I 2021-08-15 21:49:22,021] 

FrozenTrial(number=47, values=[0.8931769064526088], datetime_start=datetime.datetime(2021, 8, 15, 21, 49, 41, 647127), datetime_complete=datetime.datetime(2021, 8, 15, 21, 49, 42, 222145), params={'n_neighbors': 191}, distributions={'n_neighbors': IntUniformDistribution(high=500, low=5, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=47, state=TrialState.COMPLETE, value=None)


In [24]:
print(study_knn.best_trial.value)
print(study_knn.best_trial.params)

0.8931769064526088
{'n_neighbors': 191}


In [28]:
study_mlp.best_trial.value


0.8673973448595246

## Diagnóstico de melhor modelo

Após os testes realizados com o Random Forest, Decision Tree, Naive Bayes, MLP e KNN, encontramos os seguintes resultados para seus melhores modelos:

*   Decision Tree: 0.8971904908922507
*   Random Forest: 0.8920963260265514
*   Naive Bayes: 0.8672429762272307
*   MLP: 0.8673973448595246
*   KNN: 0.8931769064526088

Sendo assim, a Decision Tree foi a que teve a maior acurácia, com o modelo abaixo sendo o melhor encontrado, usaremos esses valores para testar contra o grupo de testes.


In [31]:
#{'max_depth': 8, 'min_samples_split': 6, 'min_samples_leaf': 8}

max_depth = 8
split     = 6
leaf      = 8

decisionTree_selection = DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=leaf, min_samples_split=split)
decisionTree_selection.fit(X_train, y_train)

y_test_prediction = decisionTree_selection.predict(X_test)
accuracy_validation = accuracy_score(y_test, y_test_prediction)

In [32]:
print("Matriz de Confusão")
print(confusion_matrix(y_test, y_test_prediction))

print("==========")

print("Acurácia")
print(accuracy_validation)

Matriz de Confusão
[[5473  370]
 [ 370  266]]
Acurácia
0.8857848433400216


## Feature importance

Verificamos o nível de relevância das colunas do dataset.